# QUERY IN MYSQL

In [1]:
import pandas as pd
import mysql.connector

# Kết nối vào MySQL
mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    password="Trantuankha0508@"
)

# Tạo database nếu chưa tồn tại
mycursor = mydb.cursor()
mycursor.execute("CREATE DATABASE IF NOT EXISTS covid_stats")
mycursor.execute("USE covid_stats")

# Tạo bảng để lưu dữ liệu từ file CSV
mycursor.execute("""
    CREATE TABLE IF NOT EXISTS covid_statistics (
        id INT AUTO_INCREMENT PRIMARY KEY,
        Country VARCHAR(255),
        TotalCases INT,
        NewCases INT,
        TotalDeaths FLOAT,
        NewDeaths FLOAT,
        TotalRecovered FLOAT,
        NewRecovered FLOAT,
        ActiveCases FLOAT,
        Serious FLOAT,
        TopCasesPer1MPop FLOAT,
        DeathsPer1MPop FLOAT,
        TotalTests FLOAT,
        TestsPer1MPop FLOAT,
        Population FLOAT
    )
""")


In [2]:
# Đọc dữ liệu từ file CSV vào DataFrame
df = pd.read_csv("C:/Users/ACER/Downloads/Covid-19.csv")

for col in df.columns:
    if df[col].dtype == 'float64' or df[col].dtype == 'int64':
        df[col] = df[col].fillna(0)  
    else:
        df[col] = df[col].astype(str).fillna('')

df = df.drop('Unnamed: 0',axis=1)

In [3]:
# Chèn dữ liệu từ DataFrame vào bảng MySQL
for index, row in df.iterrows():
    sql = """
        INSERT INTO covid_statistics (
            Country, TotalCases, NewCases, TotalDeaths, NewDeaths,
            TotalRecovered, NewRecovered, ActiveCases, Serious,
            TopCasesPer1MPop, DeathsPer1MPop, TotalTests, TestsPer1MPop, Population
        ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """
    val = (
        row['Country'], row['Total Cases'], row['New Cases'], row['Total Deaths'],
        row['New Deaths'], row['Total Recovered'], row['New Recovered'],
        row['Active Cases'], row['Serious'], row['Top Cases/1MPop'], row['Deaths/1MPop'],
        row['Total Tests'], row['Tests/1MPop'], row['Population']
    )
    mycursor.execute(sql, val)


mydb.commit()

In [8]:
mycursor.execute("SELECT * FROM covid_statistics")
result = mycursor.fetchall()

print("\nDữ liệu từ bảng covid_statistics:")
for row in result:
    print(row)


Dữ liệu từ bảng covid_statistics:
(1, 'North America', 131889132, 0, 1695940.0, 0.0, 127665000.0, 350.0, 2528060.0, 6095.0, 0.0, 0.0, 0.0, 0.0, 0.0)
(2, 'Asia', 221500265, 0, 1553660.0, 0.0, 205673000.0, 0.0, 14273500.0, 14733.0, 0.0, 0.0, 0.0, 0.0, 0.0)
(3, 'Europe', 253406198, 0, 2101820.0, 0.0, 248754000.0, 474.0, 2550270.0, 4453.0, 0.0, 0.0, 0.0, 0.0, 0.0)
(4, 'South America', 70200879, 0, 1367330.0, 0.0, 66683600.0, 0.0, 2149960.0, 8953.0, 0.0, 0.0, 0.0, 0.0, 0.0)
(5, 'Oceania', 14895771, 0, 33015.0, 0.0, 14752400.0, 0.0, 110368.0, 31.0, 0.0, 0.0, 0.0, 0.0, 0.0)
(6, 'Africa', 12860924, 0, 258892.0, 0.0, 12090800.0, 0.0, 511224.0, 529.0, 0.0, 0.0, 0.0, 0.0, 0.0)
(7, 'World', 704753890, 0, 7010680.0, 0.0, 675620000.0, 790.0, 22123400.0, 34794.0, 90413.0, 899.4, 0.0, 0.0, 0.0)
(8, 'USA', 111820082, 0, 1219490.0, 0.0, 109814000.0, 0.0, 786167.0, 940.0, 333985.0, 3642.0, 1186850000.0, 3544900.0, 334805000.0)
(9, 'India', 45035393, 0, 533570.0, 0.0, 0.0, 0.0, 0.0, 0.0, 32016.0, 379.0, 

In [4]:
# Ví dụ truy vấn: Tính tổng số ca nhiễm theo khu vực
mycursor.execute("""
    SELECT Country, SUM(TotalCases) AS TotalCases
    FROM covid_statistics
    GROUP BY Country
""")
result = mycursor.fetchall()

# In kết quả
print("\nTổng số ca nhiễm theo khu vực:")
for row in result:
    print(row)


Tổng số ca nhiễm theo khu vực:
('North America', Decimal('263778264'))
('Asia', Decimal('443000530'))
('Europe', Decimal('506812396'))
('South America', Decimal('140401758'))
('Oceania', Decimal('29791542'))
('Africa', Decimal('25721848'))
('World', Decimal('1409507780'))
('USA', Decimal('223640164'))
('India', Decimal('90070786'))
('France', Decimal('80277120'))
('Germany', Decimal('77657990'))
('Brazil', Decimal('77487836'))
('S. Korea', Decimal('69143746'))
('Japan', Decimal('67607144'))
('Italy', Decimal('53446498'))
('UK', Decimal('49820774'))
('Russia', Decimal('48248430'))
('Turkey', Decimal('34464132'))
('Spain', Decimal('27829622'))
('Australia', Decimal('23706288'))
('Vietnam', Decimal('23250390'))
('Taiwan', Decimal('20483046'))
('Argentina', Decimal('20257690'))
('Netherlands', Decimal('17271572'))
('Mexico', Decimal('15405618'))
('Iran', Decimal('15254372'))
('Indonesia', Decimal('13658442'))
('Poland', Decimal('13323982'))
('Colombia', Decimal('12800346'))
('Greece', Dec

In [5]:
# Truy vấn để tính tỷ lệ phục hồi trung bình theo từng khu vực
mycursor.execute("""
    SELECT Country, AVG(TotalRecovered / TotalCases) AS RecoveryRate
    FROM covid_statistics
    GROUP BY Country
""")
result = mycursor.fetchall()

# In ra kết quả
print("\nTỷ lệ phục hồi trung bình theo từng khu vực:")
for row in result:
    print(f"{row[0]}: {row[1]}")



Tỷ lệ phục hồi trung bình theo từng khu vực:
North America: 0.9679730699872982
Asia: 0.9285455617852195
Europe: 0.9816417828896198
South America: 0.949896709982791
Oceania: 0.9903742478318175
Africa: 0.9401196990200704
World: 0.958660675147178
USA: 0.9820635974851101
India: 0.0
France: 0.9958234675085503
Germany: 0.984846504525806
Brazil: 0.9356090419146561
S. Korea: 0.9989606290639793
Japan: 0.0
Italy: 0.9864525829175936
UK: 0.9906821600162213
Russia: 0.9760242146739283
Turkey: 0.0
Spain: 0.9890480725897032
Australia: 0.9972049609791293
Vietnam: 0.9153369900461885
Taiwan: 0.9981443189650602
Argentina: 0.9870086865777885
Netherlands: 0.9973150098902405
Mexico: 0.8957595858861358
Iran: 0.0
Indonesia: 0.9733326831859739
Poland: 0.0
Colombia: 0.9706225128601993
Greece: 0.0
Austria: 0.9956665422960633
Portugal: 0.9949933210019667
Ukraine: 0.9797736413940639
Chile: 0.9754119564638069
Malaysia: 0.9914485547341375
Canada: 0.9869031901967008
Belgium: 0.9928220507456762
Israel: 0.9910571997194

In [6]:
# Truy vấn để lấy số ca nhiễm và tử vong từng khu vực, sắp xếp theo tổng số ca nhiễm giảm dần
mycursor.execute("""
    SELECT Country, TotalCases, TotalDeaths
    FROM covid_statistics
    ORDER BY TotalCases DESC
""")
result = mycursor.fetchall()

# In ra kết quả
print("\nTop quốc gia theo tổng số ca nhiễm và tử vong:")
for row in result:
    print(f"{row[0]} - Ca nhiễm: {row[1]}, Tử vong: {row[2]}")



Top quốc gia theo tổng số ca nhiễm và tử vong:
World - Ca nhiễm: 704753890, Tử vong: 7010680.0
World - Ca nhiễm: 704753890, Tử vong: 7010680.0
Europe - Ca nhiễm: 253406198, Tử vong: 2101820.0
Europe - Ca nhiễm: 253406198, Tử vong: 2101820.0
Asia - Ca nhiễm: 221500265, Tử vong: 1553660.0
Asia - Ca nhiễm: 221500265, Tử vong: 1553660.0
North America - Ca nhiễm: 131889132, Tử vong: 1695940.0
North America - Ca nhiễm: 131889132, Tử vong: 1695940.0
USA - Ca nhiễm: 111820082, Tử vong: 1219490.0
USA - Ca nhiễm: 111820082, Tử vong: 1219490.0
South America - Ca nhiễm: 70200879, Tử vong: 1367330.0
South America - Ca nhiễm: 70200879, Tử vong: 1367330.0
India - Ca nhiễm: 45035393, Tử vong: 533570.0
India - Ca nhiễm: 45035393, Tử vong: 533570.0
France - Ca nhiễm: 40138560, Tử vong: 167642.0
France - Ca nhiễm: 40138560, Tử vong: 167642.0
Germany - Ca nhiễm: 38828995, Tử vong: 183027.0
Germany - Ca nhiễm: 38828995, Tử vong: 183027.0
Brazil - Ca nhiễm: 38743918, Tử vong: 711380.0
Brazil - Ca nhiễm: 38

In [7]:
# Truy vấn để lấy các quốc gia có tỷ lệ ca nhiễm cao hơn mức trung bình thế giới
mycursor.execute("""
    SELECT Country, TotalCases
    FROM covid_statistics
    WHERE TotalCases > (SELECT AVG(TotalCases) FROM covid_statistics)
""")
result = mycursor.fetchall()

# In ra kết quả
print("\nCác quốc gia có số ca nhiễm cao hơn mức trung bình thế giới:")
for row in result:
    print(f"{row[0]} - Ca nhiễm: {row[1]}")



Các quốc gia có số ca nhiễm cao hơn mức trung bình thế giới:
North America - Ca nhiễm: 131889132
Asia - Ca nhiễm: 221500265
Europe - Ca nhiễm: 253406198
South America - Ca nhiễm: 70200879
Oceania - Ca nhiễm: 14895771
Africa - Ca nhiễm: 12860924
World - Ca nhiễm: 704753890
USA - Ca nhiễm: 111820082
India - Ca nhiễm: 45035393
France - Ca nhiễm: 40138560
Germany - Ca nhiễm: 38828995
Brazil - Ca nhiễm: 38743918
S. Korea - Ca nhiễm: 34571873
Japan - Ca nhiễm: 33803572
Italy - Ca nhiễm: 26723249
UK - Ca nhiễm: 24910387
Russia - Ca nhiễm: 24124215
Turkey - Ca nhiễm: 17232066
Spain - Ca nhiễm: 13914811
Australia - Ca nhiễm: 11853144
Vietnam - Ca nhiễm: 11625195
Taiwan - Ca nhiễm: 10241523
Argentina - Ca nhiễm: 10128845
North America - Ca nhiễm: 131889132
Asia - Ca nhiễm: 221500265
Europe - Ca nhiễm: 253406198
South America - Ca nhiễm: 70200879
Oceania - Ca nhiễm: 14895771
Africa - Ca nhiễm: 12860924
World - Ca nhiễm: 704753890
USA - Ca nhiễm: 111820082
India - Ca nhiễm: 45035393
France - Ca n